<a href="https://colab.research.google.com/github/madmonkey7/CYSE690---Team3/blob/main/Copy_of_Phishing_Email_Detection_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!ls -l Phishing_Email.csv

-rw-r--r-- 1 root root 31457280 Mar  2 19:58 Phishing_Email.csv


In [ ]:
#import necessary libraries
import os
import pandas as pd
import numpy as np
import nltk
import re

In [ ]:
import csv
i = 0
features = []
labels = []
with open('Phishing_Email.csv', mode ='r')as file:
  csvFile = csv.reader(file)
  for line in csvFile:
    idx = line.pop(0)
    label = line.pop()
    feature = line[0]
    if label and label in ['Safe Email', 'Phishing Email']:
      features.append(feature)
      labels.append(label)
    if i == 11294:
      break
    i += 1


In [ ]:
import csv
i = 0
features = []
labels = []
with open('Phishing_Email.csv', mode = 'r') as file:
  csvFile = csv.reader(file)
  for line in csvFile:


In [ ]:
df = pd.DataFrame(zip(features, labels), columns = ['Email Text', 'Email Type'])
df

,Email Text,Email Type
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,the other side of * galicismos * * galicismo *...,Safe Email
2,re : equistar deal tickets are you still avail...,Safe Email
3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,software at incredibly low prices ( 86 % lower...,Phishing Email
...,...,...
11289,summary of sociolinguistics course syllabi dea...,Safe Email
11290,New Web Technology\nUNLIMITED WEB CONFERENCING...,Phishing Email
11291,re : anjam ' s term sheet tara : i am forwardi...,Safe Email
11292,"the ews tax update for august 31 , 2001 attach...",Safe Email


In [ ]:
#reading the dataset
df = pd.read_csv("Phishing_Email.csv", encoding='utf-8')



ParserError: Error tokenizing data. C error: EOF inside string starting at row 15160

In [ ]:
#print the dataset
df

In [ ]:
#Extracting relevant columns
selected_columns = ['Email Text', 'Email Type']
new_df = df[selected_columns].copy()

In [ ]:
new_df

In [ ]:
#drop rows with missing values
df_cleaned = new_df.dropna()
df_cleaned

In [ ]:
#printing the first 500 rows
df_cleaned.head(500)

In [ ]:
new_df['Email Type'].unique()

In [ ]:
#Removing null values from the Email Type column
new_df = new_df[(new_df['Email Type'] == 'Safe Email') | (new_df['Email Type'] == 'Phishing Email')]
new_df

In [ ]:
#counting the number of emails
new_df["Email Type"].value_counts()

In [ ]:
#printing stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
stop_words.append("n")
print(stop_words)

In [ ]:
#converting the datatype of Email Text to string
new_df['Email Text'] = new_df['Email Text'].astype(str)

In [ ]:
#converting the text to lower case
new_df['Email Text'] = new_df['Email Text'].apply(lambda x: ' '.join([word.lower() for word in x.split()]))
new_df

In [ ]:
#Replacing any non-alphanumeric characters and any numerics with a single space
for item in new_df["Email Text"]:
    res = re.sub(pattern = "[\W]", repl = " ",string= item)
    res1= re.sub(pattern = "\d", repl = " ",string= res)
    single_space = " ".join(res1.split())
    new_df["Email Text"] = new_df["Email Text"].replace(item, single_space)

In [ ]:
new_df

In [ ]:
#reading the content of email 1
new_df["Email Text"][1]

In [ ]:
#removing the stop words
new_df['Email Text'] = new_df['Email Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [ ]:
#reading the content of email 1 after removing the stop words
new_df["Email Text"][1]

In [ ]:
#importing necessary modules for stemming
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [ ]:
# Function to stem text using NLTK's PorterStemmer
stemmer = PorterStemmer()
def stem_text(text):
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

In [ ]:
# Apply the stem_text function to the 'Email Text' column and store the result in the 'stemmed_text' column
# import nltk
# nltk.download('punkt')
new_df['stemmed_text'] = new_df['Email Text'].apply(stem_text)

In [ ]:
new_df

In [ ]:
# Tokenize the 'stemmed_text' column and store the result in the 'tokens' column
new_df['tokens'] = new_df['stemmed_text'].apply(word_tokenize)

In [ ]:
new_df

In [ ]:
new_df.to_csv('cleaned.csv')

In [ ]:
#import packages for feature extraction
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
#fill missing values
df['Email Text'] = df['Email Text'].fillna('')
# Use tokenized text and prepare data for training Doc2Vec model: create TaggedDocument objects, each representing an email with tokenized text ('words') and a unique identifier ('tags').
tagged_data = [TaggedDocument(words=words.split(), tags=[str(i)]) for i, words in enumerate(df['Email Text'])]

In [ ]:
tagged_data[0]

In [ ]:
# Initialize and train the Doc2Vec model
model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=10)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
#Extract document vectors from the trained Doc2Vec model for each email represented in the TaggedDocument list.
doc_vectors = [model.dv[idx] for idx in range(len(tagged_data))]

In [ ]:
doc_vectors[0]

In [ ]:
#Reduce dimensionality of document vectors to 2D using PCA and visualize them in a scatter plot.
#Points are colored based on email type (Safe: 0, Phishing: 1).
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Get document vectors
doc_vectors = [model.dv[idx] for idx in range(len(tagged_data))]

# Reduce dimensionality to 2D
pca = PCA(n_components=2)
doc_vectors_2d = pca.fit_transform(doc_vectors)

# Create scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(doc_vectors_2d[:, 0], doc_vectors_2d[:, 1], c=[0 if 'Safe' in tag else 1 for tag in df['Email Type']], cmap='viridis')
plt.title('Document Vectors Visualization')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(label='Email Type')
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(doc_vectors,df['Email Type'], test_size=0.2, random_state=42)
